# FaultTree

## Install

Use Pkg to install the packages. The packages are located in GitHub. Please run the following commands:

- For Julia 1.0.5
```julia
using Pkg
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/DD.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/FaultTree.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/JuliaDot.jl.git"))
```

In [ ]:
using Pkg
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/DD.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/FaultTree.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/JuliaDot.jl.git"))

## Initialize

Load `FaultTree` and `JuliaDot`. `JuliaDot` provides drawing a graph of fault trees and BDD, but the package `JuliaDot` depends on your own environment. If the loding `JuliaDot` fails, please remove (comment out) loading `JuliaDot`

In [ ]:
using FaultTree
using JuliaDot

## Create Fault Trees

### Basic operations

- `ftevent`: create an event in fault tree such as a component failure. The argument is a label of event. The event is distinguished by the label.
- `*` or `&`: AND gate
- `+` or `|`: OR gate
- `!` or `~`: NOT gate
- `ftand(a, b, c, ...)`: AND gate that allows to take multiple events
- `ftor(a, b, c, ...)`: OR gate that allows to take multiple events
- `ftnot(a)`: NOT gate that is same as `!` and `~`
- `ftkofn(k, a, b, c, ...)': K-out-of-N gate where `k` is the number of events that causes the failure

### Example 1

#### Definition of component failure events

In [ ]:
x = ftevent("motor failure") # motor failure
y = ftevent("sensor failure") # sensor failure
z = ftevent("battery failure") # battery failure

#### Definition of fault tree to represent the system failure

In [ ]:
system = ftor(x, y, z)

#### Drawing a brief fault tree.

This process requires that `JuliaDot` is successfully installed.
If the install of `JuliaDot` failsed, run it and copyt the output
```julia
println(todot(system))
```
Use a graphviz like [Online Graphviz](https://dreampuf.github.io/GraphvizOnline/#digraph%20G%20%7B%0A%0A%20%20subgraph%20cluster_0%20%7B%0A%20%20%20%20style%3Dfilled%3B%0A%20%20%20%20color%3Dlightgrey%3B%0A%20%20%20%20node%20%5Bstyle%3Dfilled%2Ccolor%3Dwhite%5D%3B%0A%20%20%20%20a0%20-%3E%20a1%20-%3E%20a2%20-%3E%20a3%3B%0A%20%20%20%20label%20%3D%20%22process%20%231%22%3B%0A%20%20%7D%0A%0A%20%20subgraph%20cluster_1%20%7B%0A%20%20%20%20node%20%5Bstyle%3Dfilled%5D%3B%0A%20%20%20%20b0%20-%3E%20b1%20-%3E%20b2%20-%3E%20b3%3B%0A%20%20%20%20label%20%3D%20%22process%20%232%22%3B%0A%20%20%20%20color%3Dblue%0A%20%20%7D%0A%20%20start%20-%3E%20a0%3B%0A%20%20start%20-%3E%20b0%3B%0A%20%20a1%20-%3E%20b3%3B%0A%20%20b2%20-%3E%20a3%3B%0A%20%20a3%20-%3E%20a0%3B%0A%20%20a3%20-%3E%20end%3B%0A%20%20b3%20-%3E%20end%3B%0A%0A%20%20start%20%5Bshape%3DMdiamond%5D%3B%0A%20%20end%20%5Bshape%3DMsquare%5D%3B%0A%7D)

In [ ]:
# println(todot(system))
draw(todot(system))

#### Minimal cut set

In [ ]:
ftmcs(system)

#### Define failure probabilities for compoments

The following defintion uses the ftevent variables. But if we know the symbols for ftevent, we can use them as follows.
```julia
params = Dict(
    Symbol("motor failure") => 0.0001,
    Symbol("sensor failure") => 0.0001,
    Symbol("battery failure") => 0.0001
)
```

In [ ]:
params = Dict(
    x.var => 0.001, # failure probability for motor
    y.var => 0.0001, # failure probability for sensor
    z.var => 0.00001  # failure probability for battery
)

#### Compute the system failure probability

In [ ]:
fteval(system, params)

#### Birunbaum importance measure

In [ ]:
# define the derivative for a parameter.
dparams_x = Dict(x.var => 1.0, y.var => 0.0, z.var => 0.0)
dparams_y = Dict(x.var => 0.0, y.var => 1.0, z.var => 0.0)
dparams_z = Dict(x.var => 0.0, y.var => 0.0, z.var => 1.0)

In [ ]:
using Printf
@printf("%20s %.5f\n", x.var, fteval(system, params, dparams_x))
@printf("%20s %.5f\n", y.var, fteval(system, params, dparams_y))
@printf("%20s %.5f\n", z.var, fteval(system, params, dparams_z))

### Example 2

#### Definition of component failure events

In [ ]:
x = [ftevent("motor failure ", i) for i = ["arm1", "arm2", "arm3"]]
y = [ftevent("sensor failure ", i) for i = ["left", "right"]]
z = ftevent("battery failure")

#### Definition of fault tree to represent the system failure

In [ ]:
sub = [x[i] + z for i = 1:3]
system = ftkofn(2, sub[1], sub[2], sub[3]) + (y[1] * y[2])

#### Drawing a brief fault tree.

In [ ]:
## println(todot(system))
draw(todot(system))

In [ ]:
top, forest = bdd(system)
draw(todot(forest, top))

#### Minimal cut set

In [ ]:
ftmcs(system)

#### Define failure probabilities for compoments

In [ ]:
params = Dict(
    [x[i].var => 0.001 for i = 1:3]...,
    [y[i].var => 0.0001 for i = 1:2]...,
    z.var => 0.00001
)

#### Compute the system failure probability

In [ ]:
fteval(system, params)

#### Birunbaum importance measure

In [ ]:
# define the derivative for a parameter.
dparams_x1 = Dict(x[1].var => 1.0, x[2].var => 0.0, x[3].var => 0.0, y[1].var => 0.0, y[2].var => 0.0, z.var => 0.0)
dparams_x2 = Dict(x[1].var => 0.0, x[2].var => 1.0, x[3].var => 0.0, y[1].var => 0.0, y[2].var => 0.0, z.var => 0.0)
dparams_x3 = Dict(x[1].var => 0.0, x[2].var => 0.0, x[3].var => 1.0, y[1].var => 0.0, y[2].var => 0.0, z.var => 0.0)
dparams_y1 = Dict(x[1].var => 0.0, x[2].var => 0.0, x[3].var => 0.0, y[1].var => 1.0, y[2].var => 0.0, z.var => 0.0)
dparams_y2 = Dict(x[1].var => 0.0, x[2].var => 0.0, x[3].var => 0.0, y[1].var => 0.0, y[2].var => 1.0, z.var => 0.0)
dparams_z = Dict(x[1].var => 0.0, x[2].var => 0.0, x[3].var => 0.0, y[1].var => 0.0, y[2].var => 0.0, z.var => 1.0)

In [ ]:
# importance for x
using Printf
@printf("%20s %.5f\n", x[1].var, fteval(system, params, dparams_x1))
@printf("%20s %.5f\n", x[2].var, fteval(system, params, dparams_x2))
@printf("%20s %.5f\n", x[3].var, fteval(system, params, dparams_x3))
@printf("%20s %.5f\n", y[1].var, fteval(system, params, dparams_y1))
@printf("%20s %.5f\n", y[2].var, fteval(system, params, dparams_y2))
@printf("%20s %.5f\n", z.var, fteval(system, params, dparams_z))

### Example 3

This example is to present **a wrong case** of Example 2.
Generating the redundancy of ftevent should use different objects.
Please do not mistake it.

#### Definition of component failure events

In [ ]:
x = ftevent("motor failure")
y = ftevent("sensor failure")
z = ftevent("battery failure")

#### Definition of fault tree to represent the system failure

In [ ]:
sub = x + z
system = ftkofn(2, sub, sub, sub) + (y * y) ## sub and y are not redundant!

#### Drawing a brief fault tree.

In [ ]:
draw(todot(system))

In [ ]:
top, forest = bdd(system)
draw(todot(forest, top))

#### Minimal cut set

In [ ]:
ftmcs(system)

#### Define failure probabilities for compoments

In [ ]:
params = Dict(
    x.var => 0.001,
    y.var => 0.0001,
    z.var => 0.00001
)

#### Compute the system failure probability

In [ ]:
fteval(system, params)

#### Birunbaum importance measure

In [ ]:
# define the derivative for a parameter.
dparams_x = Dict(x.var => 1.0, y.var => 0.0, z.var => 0.0)
dparams_y = Dict(x.var => 0.0, y.var => 1.0, z.var => 0.0)
dparams_z = Dict(x.var => 0.0, y.var => 0.0, z.var => 1.0)

In [ ]:
# importance for x
using Printf
@printf("%20s %.5f\n", x.var, fteval(system, params, dparams_x))
@printf("%20s %.5f\n", y.var, fteval(system, params, dparams_y))
@printf("%20s %.5f\n", z.var, fteval(system, params, dparams_z))